In [ ]:
!pip install nfstream

In [ ]:

from nfstream import NFStreamer, NFPlugin
import pandas as pd
from google.colab import files
from datetime import datetime, timedelta
import sys

In [ ]:
class Packet40Count(NFPlugin):
    def on_init(self, packet, flow):
        if packet.ip_size == 40:
            flow.udps.packet_with_40_ip_size = 1
        else:
            flow.udps.packet_with_40_ip_size = 0

    def on_update(self, packet, flow):
        if packet.ip_size == 40:
            flow.udps.packet_with_40_ip_size += 1

In [ ]:
def summary_data(file_name):
    streamer = NFStreamer(file_name, udps=Packet40Count()).to_pandas()
    streamer.to_excel('output.xlsx')
    if "VPN" in streamer['application_category_name'].unique():
        app_bytes = streamer.groupby(['application_name', 'application_category_name'], as_index=False, sort=True)['bidirectional_bytes'].sum()
        print(f'Protocols and data for {file_name}:\n{app_bytes.to_string(index=False)}')
    else:
        print(f'No VPN traffic detected in {file_name}, only: {",".join([x for x in streamer["application_name"].unique()])}')


In [ ]:
files.upload()

In [ ]:
# 1 подпункт
sys.stdout = open('report.md', 'w')
summary_data('IKEv2.2.pcapng')

# 2 подпункт

streamer = NFStreamer('IKEv2.2.pcapng', udps=Packet40Count()).to_pandas()
data = streamer[['src_ip', 'dst_ip', 'bidirectional_packets', 'bidirectional_bytes',
            'application_name', 'application_category_name']]
print("\nВся информация:\n")
print(data)
group = streamer[['src_ip', 'dst_ip', 'application_name' ]]
print("\nГруппировка по приложению:\n")
print(group.groupby(['application_name', 'src_ip'], sort=True).first())

# 3 подпункт

time = streamer['bidirectional_first_seen_ms']
print("\nНачало захвата:\n")
print(datetime.fromtimestamp(round(time[0]/1000)))
time = streamer['bidirectional_last_seen_ms']
print("\nКонец захвата:\n")
print(datetime.fromtimestamp(round(time[len(time.index)-1] / 1000)))

# 4 подпункт
print("\nПолезная информация\n")
print(streamer.groupby(['application_name', 'application_category_name'], as_index=False, sort=True)['bidirectional_packets'].sum())

sys.stdout.close()